In [560]:
import os
import pandas as pd 
import re
import requests 
import bs4

In [561]:
os.getcwd()

'/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main'

In [562]:
def process_season_df(df):
    # Your helper function to process each CSV file
    # Example: Read the CSV and display the first few rows
    #df = pd.read_csv(file_path)

    if (len(df) == 0):
        0==0
        #when they not in the legaue 
        return df

    
    if df.shape[1] == 7:
    # Check if the word 'year' appears in any rows
        if not any(df.apply(lambda row: 'year' in row.astype(str).str.lower(), axis=1)):
            # Add a new column 'Year' at the end of the DataFrame with all values set to 0
            df['Year'] = 0
            # Add a 9th column 'bonus money' set equal to 0
            df['Bonus Money'] = 0

    # Check if the last row has the value 2023 or '2023' in it
    if '2023' in df.iloc[-1].values or "'2023'" in df.iloc[-1].values:
        # Rename columns and create a new 'bonus money' column
        df = df.rename(columns={'Unnamed: 0': 'Name',
                                'EST. BASE SALARY': 'Weekly Salary',
                                'Unnamed: 2': 'Yearly Salary',
                                'EST. BONUS': 'Inflation-Adjusted Yearly Salary',
                                'CONTRACT DETAILS': 'Position',
                                'Unnamed: 5': 'Age',
                                'Unnamed: 6': 'Nationality',
                                'Unnamed: 7': 'Season'})
        df['Season'] = df['Unnamed: 14']
        df['Nationality'] = df['Unnamed: 13']
        df['Age'] = df['Unnamed: 11']
        df['Position'] = df['BIO']
        df = df.iloc[:, 0:8]
        df['Bonus Money'] = df['Inflation-Adjusted Yearly Salary']


    else:
        # Rename columns
        df = df.rename(columns={'Unnamed: 0': 'Name',
                                'EST. BASE SALARY': 'Weekly Salary',
                                'Unnamed: 2': 'Yearly Salary',
                                'EST. BONUS': 'Inflation-Adjusted Yearly Salary',
                                'CONTRACT DETAILS': 'Position',
                                'Unnamed: 5': 'Age',
                                'Unnamed: 6': 'Nationality',
                                'Unnamed: 7': 'Season'})
        # Add a 9th column 'bonus money' set equal to 0
        df['Bonus Money'] = 0

    if(df.columns[3] == 'INLFATION (2023)'):
        df = df.rename(columns={'INLFATION (2023)': 'Inflation-Adjusted Yearly Salary'})
    if(df.columns[4] == 'BIO'):
        df = df.rename(columns={'BIO': 'Position'})
        #print(f"Processing {file_path}:")
        #print(df.head())

    if(df.columns[3] == 'INFLATION (2023)'):
        df = df.rename(columns={'INFLATION (2023)': 'Inflation-Adjusted Yearly Salary'})

    if(df.columns[4] == 'ORGANIC'):
        df = df.rename(columns={'ORGANIC': 'Position'})

    return df


In [563]:

#def process_subfolders(folder_path):
"""
Process each subfolder in the specified folder, read CSV files, and combine them into one DataFrame.

Parameters:
- folder_path (str): The path to the folder containing subfolders with CSV files.

Returns:
- subfolder_dataframes (dict): A dictionary where keys are subfolder names and values are DataFrames.
"""
def create_dict_from(folder_path):

    subfolder_dataframes = {}

    # Ensure the main folder exists
    if not os.path.exists(folder_path):
        print(f"Error: Folder '{folder_path}' does not exist.")
        #return subfolder_dataframes

    # Get a list of all subfolders in the main folder
    subfolders = [subfolder for subfolder in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, subfolder))]

    # Loop through each subfolder
    for subfolder in subfolders:
        subfolder_path = os.path.join(folder_path, subfolder)

        # Get a list of CSV files in the subfolder
        csv_files = [file for file in os.listdir(subfolder_path) if file.endswith('.csv')]
        

        # Create an empty DataFrame to store data for the current subfolder
        subfolder_dataframe = pd.DataFrame()

        # Loop through each CSV file in the subfolder
        for csv_file in csv_files:
            csv_file_path = os.path.join(subfolder_path, csv_file)

            #save the club
            club_name = csv_file_path.split('/')[12]
            season = csv_file_path.split('/')[13].split('-')[0]
            
            # Read the CSV file into a DataFrame
            df = pd.read_csv(csv_file_path)

            # Check if the DataFrame has non-zero length
            if len(df) > 0:
                df = process_season_df(df)

                if 'Year' in df.columns:
                    #print('ya')
                    df = df.rename(columns={'Year': 'Season'})
                    df['Season'] = season
                
                df = df.drop(0).reset_index(drop=True)

                for i in range(0, len(df)):
                    if(pd.isna(df.at[i, 'Bonus Money'])):
                        df.at[i, 'Bonus Money'] = 0

                for i in range(0, len(df)):
                    if((pd.isna(df.at[i, 'Inflation-Adjusted Yearly Salary'])) & (df.at[i, 'Season'] == '2023')):
                        df.at[i, 'Inflation-Adjusted Yearly Salary'] = df.at[i, 'Yearly Salary']
                    else:
                        0==0
                        #df.at[i, 'Inflation-Adjusted Yearly Salary'] = df.at[i, 'Yearly Salary']

                df['Season'] = pd.to_numeric(df['Season'], errors='coerce')
            
                # Concatenate the DataFrame to the subfolder DataFrame
                subfolder_dataframe = pd.concat([subfolder_dataframe, df], ignore_index=True)

            if 'NOT IN LEAGUE' in subfolder_dataframe.columns:
                column_name_to_drop = 'NOT IN LEAGUE'
                subfolder_dataframe = subfolder_dataframe.drop(columns=column_name_to_drop)

            if 'NOT N LEAGUE' in subfolder_dataframe.columns:
                column_name_to_drop = 'NOT N LEAGUE'
                subfolder_dataframe = subfolder_dataframe.drop(columns=column_name_to_drop)

            if 'EAST. BASIC SALARY' in subfolder_dataframe.columns:
                print(club_name, subfolder_dataframe.shape[1], season)

            if 'NO DATA' in subfolder_dataframe.columns:
                column_name_to_drop = 'NO DATA'
                subfolder_dataframe = subfolder_dataframe.drop(columns=column_name_to_drop)

        # Add the subfolder DataFrame to the dictionary
        subfolder_dataframe['Club'] = club_name
        subfolder_dataframes[subfolder] = subfolder_dataframe
    
    return subfolder_dataframes
        

    # Example usage
    #subfolder_dataframes = process_subfolders(folder_path)

In [564]:
def combine_dataframes(dictionary_of_dataframes):
    # Extract the DataFrames from the dictionary values
    dataframes_list = list(dictionary_of_dataframes.values())

    # Combine the DataFrames using pd.concat
    combined_dataframe = pd.concat(dataframes_list, ignore_index=True)

    return combined_dataframe

# WORKING

In [565]:
def get_subfolders(folder_path):
    subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    return subfolders

# Example usage:
folder_path = '/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/work/analysis work 9/soccer/Too Much Talent/Rouel Salary New Data/'

subfolders = get_subfolders(folder_path)
#subfolders = subfolders[1:]
subfolders

['Turkish Süper Lig',
 '1. Bundesliga',
 'Portuguese Primeira Liga',
 'Dutch Eredivisie',
 'Italian Serie A',
 'English Premier League',
 'French Ligue 1',
 'Spanish Liga Santander']

In [566]:
grand_final_df = pd.DataFrame()

for folder in subfolders:
    print(folder)
    folder_path_ger = f"/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/work/analysis work 9/soccer/Too Much Talent/Rouel Salary New Data/{folder}/"
    subfolder_ger = create_dict_from(folder_path_ger)
    result_dataframe = combine_dataframes(subfolder_ger)
    result_dataframe['League'] = folder
    grand_final_df = pd.concat([grand_final_df, result_dataframe], ignore_index=True)




Turkish Süper Lig
1. Bundesliga
Portuguese Primeira Liga
Dutch Eredivisie
Italian Serie A
English Premier League
French Ligue 1
Spanish Liga Santander


In [567]:
grand_final_df.to_csv('capology_league_data_checking.csv', index=False)

# DEBUGGING

## WHOLE FOLDER

In [552]:
folder_path = f"/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/work/analysis work 9/soccer/Too Much Talent/Rouel Salary New Data/English Premier League"

#folder_name = ['Dutch Eredivisie']
subfolder_dataframes = {}

# Ensure the main folder exists
if not os.path.exists(folder_path):
    print(f"Error: Folder '{folder_path}' does not exist.")
    #return subfolder_dataframes

# Get a list of all subfolders in the main folder
subfolders = [subfolder for subfolder in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, subfolder))]

# Loop through each subfolder
for subfolder in subfolders:
    #ONE TEAM NAME FILE PATH
    subfolder_path = os.path.join(folder_path, subfolder)

    # Get a list of CSV files FOR ONE TEAM
    csv_files = [file for file in os.listdir(subfolder_path) if file.endswith('.csv')]
    

    # Create an empty DataFrame to store data for the current subfolder
    subfolder_dataframe = pd.DataFrame()

    # Loop through each CSV file 
    for csv_file in csv_files:
        csv_file_path = os.path.join(subfolder_path, csv_file)

        #save the club
        club_name = csv_file_path.split('/')[12]
        season = csv_file_path.split('/')[13].split('-')[0]
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_file_path)

        # Check if the DataFrame has non-zero length
        if len(df) > 0:
            df = process_season_df(df)

            if 'Year' in df.columns:
                #print('ya')
                df = df.rename(columns={'Year': 'Season'})
                df['Season'] = season
            
            df = df.drop(0).reset_index(drop=True)

            for i in range(0, len(df)):
                if(pd.isna(df.at[i, 'Bonus Money'])):
                    df.at[i, 'Bonus Money'] = 0

            for i in range(0, len(df)):
                if((pd.isna(df.at[i, 'Inflation-Adjusted Yearly Salary'])) & (df.at[i, 'Season'] == '2023')):
                    df.at[i, 'Inflation-Adjusted Yearly Salary'] = df.at[i, 'Yearly Salary']
                else:
                    0==0
                    #df.at[i, 'Inflation-Adjusted Yearly Salary'] = df.at[i, 'Yearly Salary']

            df['Season'] = pd.to_numeric(df['Season'], errors='coerce')
        
            # Concatenate the DataFrame to the subfolder DataFrame
        subfolder_dataframe = pd.concat([subfolder_dataframe, df], ignore_index=True)

        if 'NOT IN LEAGUE' in subfolder_dataframe.columns:
            column_name_to_drop = 'NOT IN LEAGUE'
            subfolder_dataframe = subfolder_dataframe.drop(columns=column_name_to_drop)

        if 'NOT N LEAGUE' in subfolder_dataframe.columns:
            column_name_to_drop = 'NOT N LEAGUE'
            subfolder_dataframe = subfolder_dataframe.drop(columns=column_name_to_drop)

        if 'EAST. BASIC SALARY' in subfolder_dataframe.columns:
            print(club_name, subfolder_dataframe.shape[1], season)

        if 'NO DATA' in subfolder_dataframe.columns:
            column_name_to_drop = 'NO DATA'
            subfolder_dataframe = subfolder_dataframe.drop(columns=column_name_to_drop)
            #print(club_name, subfolder_dataframe.shape[1], season)
            

        # if((df.shape[1] != 1) & (df.shape[1] != 9)):
        #     print(club_name, df.shape[1], season)
        if((subfolder_dataframe.shape[1] != 0) & (subfolder_dataframe.shape[1] != 9)):
            print(club_name, subfolder_dataframe.shape[1], season)

    # Add the subfolder DataFrame to the dictionary
    subfolder_dataframe['Club'] = club_name
    subfolder_dataframes[subfolder] = subfolder_dataframe
    #print(club_name, subfolder_dataframe.shape[1])


result_dataframe_big = combine_dataframes(subfolder_dataframes)
result_dataframe_big['League'] = csv_file_path.split('/')[11]

In [553]:
#subfolder_dataframes['Ankaragucu']
result_dataframe_big#.head(5)#.shape[1]

,Name,Weekly Salary,Yearly Salary,Inflation-Adjusted Yearly Salary,Position,Age,Nationality,Season,Bonus Money,Club,League
0,Brandon Williams,74638,3881186,3881186,D,21,England,2021.0,0.0,Norwich,English Premier League
1,Ozan Kabak,68897,3582634,3582634,D,21,Turkey,2021.0,0.0,Norwich,English Premier League
2,Milot Rashica,51673,2686975,2686975,F,25,Kosovo,2021.0,0.0,Norwich,English Premier League
3,Ben Gibson,45931,2388422,2388422,D,29,England,2021.0,0.0,Norwich,English Premier League
4,Pierre Lees-Melou,35773,1860214,1860214,M,28,France,2021.0,0.0,Norwich,English Premier League
...,...,...,...,...,...,...,...,...,...,...,...
7483,Jason Gilchrist,508,26410,31814,F,19,England,2014.0,0,Burnley,English Premier League
7484,Cameron Dummigan,459,23884,28771,D,18,Northern Ireland,2014.0,0,Burnley,English Premier League
7485,Nathaniel Chalobah,NaN,NaN,NaN,M,19,England,2014.0,0,Burnley,English Premier League
7486,Matthew Taylor,NaN,NaN,NaN,M,32,England,2014.0,0,Burnley,English Premier League


## One team all years

In [544]:
#test_subfolder = subfolders[9] antalyaspor
test_subfolder = subfolders[13]
k = 0

for subfolder in test_subfolder:
    subfolder_path = os.path.join(folder_path, test_subfolder)

    # Get a list of CSV files in the subfolder
    csv_files = [file for file in os.listdir(subfolder_path) if file.endswith('.csv')]
    

    # Create an empty DataFrame to store data for the current subfolder
    subfolder_dataframe = pd.DataFrame()

    # Loop through each CSV file in the subfolder
    for csv_file in csv_files:
        csv_file_path = os.path.join(subfolder_path, csv_file)

        #save the club
        club_name = csv_file_path.split('/')[12]
        season = csv_file_path.split('/')[13].split('-')[0]
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_file_path)

        # Check if the DataFrame has non-zero length
        if len(df) > 0:
            df = process_season_df(df)

            if 'Year' in df.columns:
                #print('ya')
                df = df.rename(columns={'Year': 'Season'})
                df['Season'] = season
            
            df = df.drop(0).reset_index(drop=True)

            for i in range(0, len(df)):
                if(pd.isna(df.at[i, 'Bonus Money'])):
                    df.at[i, 'Bonus Money'] = 0

            for i in range(0, len(df)):
                if((pd.isna(df.at[i, 'Inflation-Adjusted Yearly Salary'])) & (df.at[i, 'Season'] == '2023')):
                    df.at[i, 'Inflation-Adjusted Yearly Salary'] = df.at[i, 'Yearly Salary']
                else:
                    0==0
                    #df.at[i, 'Inflation-Adjusted Yearly Salary'] = df.at[i, 'Yearly Salary']

            df['Season'] = pd.to_numeric(df['Season'], errors='coerce')
            
        
            # Concatenate the DataFrame to the subfolder DataFrame
        subfolder_dataframe = pd.concat([subfolder_dataframe, df], ignore_index=True)

        if 'NOT IN LEAGUE' in subfolder_dataframe.columns:
            column_name_to_drop = 'NOT IN LEAGUE'
            subfolder_dataframe = subfolder_dataframe.drop(columns=column_name_to_drop)
            #subfolder_dataframe = subfolder_dataframe.drop(subfolder_dataframe.columns[(subfolder_dataframe.shape[1]) - 1], axis=1)

        if((df.shape[1] != 1) & (df.shape[1] != 9)):
            print(club_name, df.shape[1], season)




In [545]:
subfolder_dataframe

,Name,Weekly Salary,Yearly Salary,Inflation-Adjusted Yearly Salary,Position,Age,Nationality,Season,Bonus Money
0,Eden Hazard,"€600,962","€31,250,000","€33,872,055",F,29,Belgium,2020,0
1,Sergio Ramos,"€468,846","€24,380,000","€26,425,623",D,34,Spain,2020,0
2,Marcelo,"€468,846","€24,380,000","€26,425,623",D,32,Brazil,2020,0
3,Toni Kroos,"€468,846","€24,380,000","€26,425,623",M,30,Germany,2020,0
4,Luka Modric,"€420,769","€21,880,000","€23,715,858",M,35,Croatia,2020,0
...,...,...,...,...,...,...,...,...,...
330,Daniel Carvajal,NaN,NaN,NaN,D,22,Spain,2014,0
331,Chicharito,NaN,NaN,NaN,F,26,Mexico,2014,0
332,Ángel Di María,NaN,NaN,NaN,F,26,Argentina,2014,0
333,Martin Ödegaard,NaN,NaN,NaN,F,15,Norway,2014,0


## SINGLE YEAR

In [395]:
#df = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/work/analysis work 9/soccer/Too Much Talent/Rouel Salary New Data/Italian Serie A/Bologna/2019-2020.csv')
#df = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/work/analysis work 9/soccer/Too Much Talent/Rouel Salary New Data/1. Bundesliga/Bayern Munich/2023-2024.csv')
df_testing = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/work/analysis work 9/soccer/Too Much Talent/Rouel Salary New Data/Turkish Süper Lig/Galatasaray/2013-2014.csv')

test_seasonnum = '2013'

In [396]:
if df_testing.shape[1] == 7:
    # Check if the word 'year' appears in any rows
    if not any(df_testing.apply(lambda row: 'year' in row.astype(str).str.lower(), axis=1)):
        # Add a new column 'Year' at the end of the DataFrame with all values set to 0
        df_testing['Year'] = 0
        # Add a 9th column 'bonus money' set equal to 0
        df_testing['Bonus Money'] = 0

# Check if the last row has the value 2023 or '2023' in it
if '2023' in df_testing.iloc[-1].values or "'2023'" in df_testing.iloc[-1].values:
    # Rename columns and create a new 'bonus money' column
    df_testing = df_testing.rename(columns={'Unnamed: 0': 'Name',
                            'EST. BASE SALARY': 'Weekly Salary',
                            'Unnamed: 2': 'Yearly Salary',
                            'EST. BONUS': 'Inflation-Adjusted Yearly Salary',
                            'CONTRACT DETAILS': 'Position',
                            'Unnamed: 5': 'Age',
                            'Unnamed: 6': 'Nationality',
                            'Unnamed: 7': 'Season'})
    df_testing['Season'] = df_testing['Unnamed: 14']
    df_testing['Nationality'] = df_testing['Unnamed: 13']
    df_testing['Age'] = df_testing['Unnamed: 11']
    df_testing['Position'] = df_testing['BIO']
    df_testing = df_testing.iloc[:, 0:8]
    df_testing['Bonus Money'] = df_testing['Inflation-Adjusted Yearly Salary']


else:
    # Rename columns
    df_testing = df_testing.rename(columns={'Unnamed: 0': 'Name',
                            'EST. BASE SALARY': 'Weekly Salary',
                            'Unnamed: 2': 'Yearly Salary',
                            'EST. BONUS': 'Inflation-Adjusted Yearly Salary',
                            'CONTRACT DETAILS': 'Position',
                            'Unnamed: 5': 'Age',
                            'Unnamed: 6': 'Nationality',
                            'Unnamed: 7': 'Season'})
    # Add a 9th column 'bonus money' set equal to 0
    df_testing['Bonus Money'] = 0

if(df_testing.columns[3] == 'INLFATION (2023)'):
    df_testing = df_testing.rename(columns={'INLFATION (2023)': 'Inflation-Adjusted Yearly Salary'})
if(df_testing.columns[4] == 'BIO'):
    df_testing = df_testing.rename(columns={'BIO': 'Position'})

if(df_testing.columns[3] == 'INFLATION (2023)'):
    df_testing = df_testing.rename(columns={'INFLATION (2023)': 'Inflation-Adjusted Yearly Salary'})

if(df_testing.columns[4] == 'ORGANIC'):
    df_testing = df_testing.rename(columns={'ORGANIC': 'Position'})

In [397]:
df_testing = df_testing.drop(0).reset_index(drop=True)

if 'Year' in df_testing.columns:
    print('ya')
    df_testing = df_testing.rename(columns={'Year': 'Season'})
    df_testing['Season'] = test_seasonnum

for i in range(0, len(df_testing)):
    if(pd.isna(df_testing.at[i, 'Bonus Money'])):
        df_testing.at[i, 'Bonus Money'] = 0

for i in range(0, len(df_testing)):
    if((pd.isna(df_testing.at[i, 'Inflation-Adjusted Yearly Salary'])) & (df_testing.at[i, 'Season'] == '2023')):
        df_testing.at[i, 'Inflation-Adjusted Yearly Salary'] = df_testing.at[i, 'Yearly Salary']
    else:
        0==0
        #df.at[i, 'Inflation-Adjusted Yearly Salary'] = df.at[i, 'Yearly Salary']

df_testing['Season'] = pd.to_numeric(df_testing['Season'], errors='coerce')


In [398]:
df_testing.head()

,Name,EAST. BASIC SALARY,Yearly Salary,Inflation-Adjusted Yearly Salary,Position,Age,Nationality,Season,Bonus Money
0,Didier drogba,85000,4420000,17673258,F,35,Ivory Coast,2013,0
1,Wesley Sneijder,68077,3540000,14154600,M,29,Netherlands,2013,0
2,Felipe Melo,65962,3430000,13714768,M,30,Brazil,2013,0
3,Hamit Altintop,61538,3200000,12795119,M,30,Turkey,2013,0
4,Burak Yilmaz,48846,2540000,10156126,F,27,Turkey,2013,0


# OLD 

In [186]:
def process_club_folder(folder_path):
    # List all CSV files in the folder
    grand_dataframe_final = pd.DataFrame()
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    # Process each CSV file in the folder
    for csv_file in csv_files:
        csv_file_path = os.path.join(folder_path, csv_file) 
        club_df = pd.read_csv(csv_file_path)
        if(len(club_df) == 0):
            #this is when they are not in the league that year 
            0==0
        else:    
            #and the other one
            club_df = process_season_csv(csv_file_path)

            grand_dataframe_final = pd.concat([grand_dataframe_final, club_df], ignore_index=True)

    return grand_dataframe_final

In [194]:
#test1 = '/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/work/analysis work 9/soccer/Too Much Talent/Rouel Salary New Data/French Ligue 1/Metz/2015-2016.csv'
#test1 = process_season_csv(test1)
folderpath_test = '/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/work/analysis work 9/soccer/Too Much Talent/Rouel Salary New Data/1. Bundesliga/Heidenheim/'
folderpath_test = process_club_folder(folderpath_test)
folderpath_test['Season'].value_counts()

Season
2023    28
Name: count, dtype: int64

In [ ]:
#logic for function


#create a "big" dataframe which is the target

#given a folder (Bologna) loop through all the CSVs in there 

#for each of the CSVs, call it a dataframe 
#run the function on it 

#concat it with the big DF 